<a href="https://colab.research.google.com/github/nic-fp/ResearchNotebooks/blob/master/ResnetFromScratchWay2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from keras import layers
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from keras.models import Model, load_model
from keras.preprocessing import image
from keras.utils import layer_utils, to_categorical
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
import pydot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
#from resnets_utils import *
from keras.initializers import glorot_uniform
import scipy.misc
from matplotlib.pyplot import imshow
%matplotlib inline


import keras.backend as K
K.set_image_data_format('channels_last')
K.set_learning_phase(1)



def identityBlock(X,numberOfFilters,filters):
  shortcut = X
  F1,F2,F3 = filters
  
  #first component of main path
  X = Conv2D(filters = F1,kernel_size=(1,1), strides= (1,1))(X)
  X = BatchNormalization()(X)
  X = Activation('relu')(X)

  #2nd component of main path
  X = Conv2D(filters = F2,kernel_size=(numberOfFilters,numberOfFilters), strides = (1,1), padding = 'same')(X)
  X = BatchNormalization()(X)
  X = Activation('relu')(X)

  #3rd comp
  X= Conv2D(filters = F3,kernel_size=(1,1))(X)
  X = BatchNormalization()(X)

  #Add shortcut and main

  X = Add()([X,shortcut])

  X = Activation('relu')(X)
  return X

def convolutionalBlock(X, numberOfFilters, filters, s=2):
  shortcut = X
  F1,F2,F3 = filters
  #1st comp main
  X = Conv2D(F1,kernel_size=(1,1), strides = (s,s))(X)
  X = BatchNormalization()(X)
  X = Activation('relu')(X)

  #2nd component of main path
  X = Conv2D(F2,kernel_size=(numberOfFilters,numberOfFilters), strides = (1,1), padding = 'same')(X)
  X = BatchNormalization()(X)
  X = Activation('relu')(X)

  #3rd comp
  X= Conv2D(F3, kernel_size = (1,1))(X)
  X = BatchNormalization()(X)


  #Shortcut path
  shortcut = Conv2D(F3, kernel_size= (1,1), strides = (s,s))(shortcut)
  shortcut = BatchNormalization()(shortcut)
  #add and activate
  X = Add()([X,shortcut])
  X = Activation('relu')(X)
  return X

#Make resnet50

def Resnet50(input_shape= (32,32,3), classes = 10):
  X_input = Input(shape= input_shape)

  X = ZeroPadding2D((3,3))(X_input)

  #stage 1
  X = Conv2D(64, kernel_size= (7,7), strides = (2,2))(X)
  X = BatchNormalization()(X)
  X = Activation('relu')(X)
  X = MaxPooling2D((3,3), strides = (2,2))(X)

  #Stage 2
  X = convolutionalBlock(X,3,[64,64,256])
  X = identityBlock(X,3,[64,64,256])
  X = identityBlock(X,3,[64,64,256])

  #Stage 3
  X = convolutionalBlock(X,3,[128,128,512])
  X = identityBlock(X,3,[128,128,512])
  X = identityBlock(X,3,[128,128,512])
  X = identityBlock(X,3,[128,128,512])

  #stage 4
  X = convolutionalBlock(X,3,[256,256,1024])
  X = identityBlock(X,3,[256,256,1024])
  X = identityBlock(X,3,[256,256,1024])
  X = identityBlock(X,3,[256,256,1024])
  X = identityBlock(X,3,[256,256,1024])
  X = identityBlock(X,3,[256,256,1024])

  #stage 5
  X = convolutionalBlock(X,3,[512,512,2048])
  X = identityBlock(X,3,[512,512,2048])
  X = identityBlock(X,3,[512,512,2048])

  #Final
  X = AveragePooling2D((1,1))(X)
  X = Flatten()(X)
  X = Dense(528, activation = 'relu')(X)
  X = Dense(1024, activation = 'relu')(X)
  X = Dense(classes,activation='softmax')(X)

  model = Model(inputs = X_input, outputs = X)
  
  
  return model











  

Using TensorFlow backend.


In [2]:
from keras.datasets import cifar10




(x_train, y_train), (x_test, y_test) = cifar10.load_data()

x_train = x_train.reshape((50000,32,32,3))
x_test = x_test.reshape((10000,32,32,3))

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)
# 69% on cifar 10 after 30 epochs 37s per
model = Resnet50(input_shape=(32,32,3),classes = 10)
model.compile(optimizer='adam',loss= 'categorical_crossentropy',metrics=['accuracy'])
model.fit(x_train,y_train,batch_size= 80, epochs=100,validation_split=.2)


170500096/170498071 [==============================] - 11s 0us/step













Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 40000 samples, validate on 10000 samples
Epoch 1/100
40000/40000 [==============================] - 53s 1ms/step - loss: 1.9330 - acc: 0.3040 - val_loss: 1.6210 - val_acc: 0.4062
Epoch 2/100
40000/40000 [==============================] - 39s 987us/step - loss: 1.5148 - acc: 0.4454 - val_loss: 1.4505 - val_acc: 0.4786
Epoch 3/100
40000/40000 [==============================] - 39s 985us/step - loss: 1.3624 - acc: 0.5091 - val_loss: 1.4647 - val_acc: 0.4769
Epoch 4/100
40000/40000 [==============================] - 39s 986us/step - loss: 1.2754 - acc: 0.5427 - val_loss: 1.2559 - val_acc: 0.5623
Epoch 5/100
40000/40000 [==============================] - 39s 979us/step - loss: 1.1712 - acc: 0.5835 - val_loss: 1.3366 - val_acc: 0.5352
Epoch 6/100
40000/40000 [==============================] - 38s 961us/step -

KeyboardInterrupt: ignored

In [5]:
from keras.datasets import mnist
from keras.utils import to_categorical
from keras.layers import Dense, Conv2D, Flatten,MaxPooling2D, Dropout,BatchNormalization

(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.reshape((60000,28,28,1))
x_test = x_test.reshape((10000,28,28,1))

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)


#97% accuracy after 4 epochs 55s per
model = Resnet50(input_shape=(28,28,1),classes=10)


model.compile(optimizer='adam',loss= 'categorical_crossentropy',metrics=['accuracy'])

model.fit(x_train,y_train,batch_size= 80, epochs=4,validation_split=.2)

11493376/11490434 [==============================] - 0s 0us/step
Train on 48000 samples, validate on 12000 samples
Epoch 1/4
48000/48000 [==============================] - 128s 3ms/step - loss: 0.4897 - acc: 0.8663 - val_loss: 0.1826 - val_acc: 0.9443
Epoch 2/4
48000/48000 [==============================] - 115s 2ms/step - loss: 0.2167 - acc: 0.9375 - val_loss: 0.1507 - val_acc: 0.9551
Epoch 3/4
17680/48000 [==========>...................] - ETA: 1:10 - loss: 0.1439 - acc: 0.9568

KeyboardInterrupt: ignored